In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
%matplotlib inline

In [3]:
df_Train=pd.read_csv('Train_New.csv')
df_Test=pd.read_csv('Test_New.csv')

In [4]:
df_Train.head()

,Package Name,Package Type,D1,D2,D3,D4,D5,N1,N2,N3,...,SP12,SP13,SP14,SP15,SP16,SP17,SP18,SP19,Cancellation Rules,Per Person Price
0,422,4,79,118,54,9,30,1,2,2,...,166,136,108,93,65,46,27,18,5,11509.0
1,1152,3,107,95,90,49,30,1,2,1,...,166,136,108,93,65,46,27,18,0,22485.5
2,1437,2,113,24,69,49,30,2,1,0,...,166,136,108,93,65,46,27,18,0,12421.5
3,487,3,71,68,0,33,30,2,1,1,...,166,136,108,93,65,46,27,18,0,35967.0
4,196,3,86,13,69,49,30,4,3,0,...,166,136,108,93,65,46,27,18,0,25584.0


In [5]:
df_Test.head()

,Package Name,Package Type,D1,D2,D3,D4,D5,N1,N2,N3,...,SP11,SP12,SP13,SP14,SP15,SP16,SP17,SP18,SP19,Cancellation Rules
0,841,2,35,94,69,49,30,5,0,0,...,194,166,136,108,93,65,46,27,18,0
1,1061,2,10,94,69,49,30,2,0,0,...,194,166,136,108,93,65,46,27,18,9
2,2127,4,71,94,69,49,30,3,0,0,...,194,166,136,108,93,65,46,27,18,9
3,1471,1,71,94,69,49,30,3,0,0,...,194,166,136,108,93,65,46,27,18,9
4,1938,2,21,83,92,1,30,1,2,1,...,194,166,136,108,93,65,46,27,18,0


In [6]:
print(df_Train.shape)
print(df_Test.shape)

(21000, 49)
(9000, 48)


In [7]:
df_Train.drop('PC1', axis=1, inplace=True)
df_Train.drop('PC2', axis=1, inplace=True)
df_Train.drop('PC3', axis=1, inplace=True)
df_Train.drop('PC4', axis=1, inplace=True)
df_Train.drop('PC5', axis=1, inplace=True)
df_Train.drop('Meals', axis=1, inplace=True)

In [8]:
df_Test.drop('PC1', axis=1, inplace=True)
df_Test.drop('PC2', axis=1, inplace=True)
df_Test.drop('PC3', axis=1, inplace=True)
df_Test.drop('PC4', axis=1, inplace=True)
df_Test.drop('PC5', axis=1, inplace=True)
df_Test.drop('Meals', axis=1, inplace=True)

In [9]:
df_Train.drop('N5', axis=1, inplace=True)
df_Test.drop('N5', axis=1, inplace=True)

In [10]:
df_Train.drop('A5', axis=1, inplace=True)
df_Test.drop('A5', axis=1, inplace=True)

df_Train.drop('Month', axis=1, inplace=True)
df_Test.drop('Month', axis=1, inplace=True)

In [11]:
#Adding all features except Target column
feature_cols = [c for c in df_Train.columns if c != "Per Person Price"]

In [12]:
X=df_Train.drop(['Per Person Price'],axis=1)
y=df_Train[['Per Person Price']]

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.18,random_state=33)

In [14]:
import optuna
from math import sqrt
from sklearn.ensemble import RandomForestRegressor

In [16]:
# lgb_train = RandomForestRegressor.Dataset(X_train, y_train)
# lgb_valid = RandomForestRegressor.Dataset(X_test, y_test, reference=lgb_train)

In [17]:
from sklearn.metrics import mean_squared_log_error

In [40]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1400, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 244, 388, 533, 677, 822, 966, 1111, 1255, 1400], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 14, 18, 23, 27, 32, 36, 41, 45, 50, None], 'min_samples_split': [2, 3, 5, 10], 'min_samples_leaf': [1, 2, 3, 4], 'bootstrap': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=33, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  4.8min


In [ ]:
rf_random.best_params_